In [1]:
import numpy
from scipy import linalg, special
from scipy.integrate import solve_ivp
from matplotlib import pyplot
import matplotlib

#matplotlib.rc("font", **{"family":  "serif", "weight": "normal", "size": 20})

pyplot.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Computer Modern Roman"],
    "font.size": "28",
    "text.latex.preamble": "\\usepackage[utf8]{inputenc}\\usepackage[T1]{fontenc}\\usepackage{lmodern}"
})

from tqdm.notebook import tqdm

import joblib

from born_markov import *

In [ ]:
e_up = 0.1 # eV
e_down = 0.2 # eV
U = 0.2 # eV
Gamma = 0.01 # eV
T = 300 # K
voltages = numpy.linspace(-1.5, 1.5, 501)

currents = numpy.zeros((len(voltages), 2))

for i, voltage in enumerate(tqdm(voltages)):
    solver = create_anderson_solver(e_up, e_down, U, Gamma, -0.5*voltage, 0.5*voltage, T, T)
    rho_ss, L = solver.find_steady_state() # (numpy.diag((1, 0, 0, 0)), 25, dt=1e-2)
    currents[i] = solver.get_current(rho_ss)
    
pyplot.figure(figsize=(15,10))
pyplot.plot(voltages, currents[:,0], label="$J_L$")
pyplot.plot(voltages, currents[:,1], label="$J_R$")

pyplot.grid()
pyplot.legend(loc=1)
pyplot.xlabel("$\\Delta\\phi$ [V]")
pyplot.ylabel("$J_K$ [mA]")
#pyplot.savefig("anderson_currents.pdf")
pyplot.show()

numpy.save("AN_300K", currents)

In [ ]:
pyplot.figure(figsize=(15,10))
pyplot.plot(voltages, 1e3*numpy.load("AN_100K.npy")[:,0], label="$T$=100K", c="tab:blue") 
pyplot.plot(voltages, 1e3*numpy.load("AN_100K.npy")[:,1], c="tab:blue", ls="--")
pyplot.plot(voltages, 1e3*numpy.load("AN_200K.npy")[:,0], label="$T$=200K", c="tab:orange")
pyplot.plot(voltages, 1e3*numpy.load("AN_200K.npy")[:,1], c="tab:orange", ls="--")
pyplot.plot(voltages, 1e3*numpy.load("AN_300K.npy")[:,0], label="$T$=300K", c="tab:green")
pyplot.plot(voltages, 1e3*numpy.load("AN_300K.npy")[:,1], c="tab:green", ls="--")
pyplot.grid()
pyplot.legend(loc=5, framealpha=1.)
pyplot.xlabel("$\\Delta\\phi$ [V]")
pyplot.ylabel("$J_K$ [µA]")
pyplot.savefig("AN_currents.pdf")
pyplot.show()